# PX915 Individual Project Reproducible Result - Ben Gosling

#### Import Modules for Plasma Parameter Calculation and GP regression

In [1]:
# import epoch calculators
from multiprocessing import Pool
import numpy as np
import sdf
from scipy import constants

from Py_scripts.run_epoch import * # stores functions which aid in the running of epoch simulations
from Py_scripts.sim_setup import * # stores functions which aid in the creation of setting up epoch runs

# Gaussian process regression scripts
from Py_scripts.gp import * # stores functions for performing GP regression for 1D input space

In [ ]:
mu0 = constants.mu_0
pi = np.pi
pico = 1e-12
micron = 1e-6

EPOCH requires you to specify an output directory which stores the input file to set up the simulation and store the output files. The python function below is used to create a directory within epoch_surra and populate it with one of the example input decks in the input_decks directory. 

In [ ]:
# set name of the output directory
dir = 'Data_epoch'
sub_dirs = [f'Data_{i}' for i in range(1, 11)]
dirs = [f'Data_epoch/Data_{i}' for i in range(1,11)]
# input file/setup used throughout the report
input_file = 'example_input.deck'
# set initial laser intensity in W/cm^2 (varies between 1e14 - 1e16 in the report)
intensity = 4e15 # set initial laser intensity in W/cm^2 
# set density scale length in m (varies between 300e-6 - 100e-6 in the report) 
dens_scale_len = 500 * micron
# set the number of particles per cell (set to 2048 in the report)
# set to 100 to save time
ppc = 100

# For this example input deck, the number of timesteps and grid 
# cells are fixed at 4001 and 6473 respectively (see in example_input_deck)
nx = 6473
timesteps = 4001

t_end = 2.0 * pico

In [ ]:
for i in range(len(sub_dirs)):
    epoch_sim_sub_dir(dir = dir, sub_dir= sub_dirs[i], input_file = input_file, I = intensity, Ln = dens_scale_len, ppc = ppc)

In [ ]:
def run_epoch_in_parrallel():
    pool = Pool(processes=len(dirs))
    pool.map(run_epoch, dirs)

if __name__ == '__main__':
    run_epoch_in_parrallel()

In [ ]:
def get_2D_Ey_field(dir):
        # create space-time electric field array
        Ey = np.zeros((nx, timesteps))
        for i in range(timesteps):
            fname = f'{dir}/fields_'+str(i).zfill(4)+'.sdf'
            data = sdf.read(fname, dict = True)
            Ey[:, i] = data['Electric Field/Ey'].data
        return Ey
        
def get_2D_Bz_field(dir):
        # create space-time electric field array
        Bz = np.zeros((nx, timesteps))
        for i in range(timesteps):
            fname = f'{dir}/fields_'+str(i).zfill(4)+'.sdf'
            data = sdf.read(fname, dict = True)
            Bz[:, i] = data['Magnetic Field/Bz'].data
        return Bz

In [ ]:
def winsincFIR(omega_c,omega_s,M):
    # cutoff frequency shoudl be a fraction of sampling frequency
    ker = np.sinc((omega_c / omega_s) * (np.arange(M) - (M - 1)/2))
    # Blackman window used for smooting filter
    ker *= np.blackman(M)
    # unit gain at zero frequency 
    ker /= np.sum(ker) 
    return ker

def bandpass(w0,bw,omega_s,M):
    # Angular frequency used for NIF Laser
    omega = 5.36652868179e+15
    w0 = w0 * omega
    bw = bw * omega
    # upper and lower bound frequencies of bandpass
    ub = w0 + (bw / 2)
    lb = w0 - (bw / 2)
    # create high-pass filter with cutoff at the lower-bound
    # inverse low-pass filter
    hhpf = -1 * winsincFIR(lb,omega_s,M) 
    hhpf[(M - 1) // 2] += 1
    # create low-pass filter with cutoff at the upper-bound
    hlpf = winsincFIR(ub,omega_s,M)
    # convolve the two into a band-pass filter
    h = np.convolve(hlpf, hhpf)
    return h


In [ ]:
def get_filtered_signals(dir, laser = False):
        # required fields
        Ey = get_2D_Ey_field(dir) # Ey(x,t) field
        Bz = get_2D_Bz_field(dir) # Bz(x,t) field

        n,m = Ey.shape # array size
        omega_0 = 1.0 # normalised laser frequency 
        omega_bw = 0.3 # bandswidth centred at laser frequency
        T_end = t_end # sim end time
        N = timesteps # number of time steps
        dt = T_end/N # time step
        omegaNyq = pi/dt # Nyquist Frequency
        omega_s = 2*pi/dt # sampling frequency 
        M = 1001 # half length of the filter kernel (must be odd) 

        h = bandpass(omega_0,omega_bw,omegaNyq,M) #bandpass filter

       
        # Laser signals
        Ey_laser = np.zeros((n, m))
        Bz_laser = np.zeros((n, m))

        # SRS signals
        Ey_SRS = np.zeros((n, m))
        Bz_SRS = np.zeros((n, m))

        # Fill arrays with data
        for i in range(n):
            # laser signals
            Ey_laser[i, :] = np.convolve(Ey[i,:],h,mode='same')
            Bz_laser[i, :] = np.convolve(Bz[i,:],h,mode='same')
            # SRS signals
            Ey_SRS[i, :] = Ey[i,:] - Ey_laser[i,:]
            Bz_SRS[i, :] = Bz[i,:] - Bz_laser[i,:]

        if laser:    
            return Ey_laser, Bz_laser
        else:    
            return Ey_SRS, Bz_SRS


In [ ]:
def get_bsrs(dir, ncells = 10, refelctivity = True):
    # get required field signals
    Ey, Bz = get_filtered_signals(dir, laser = False)           
    W_cm2 = 1e4 # Convert to W_cm2
    factor = mu0*W_cm2 # Denominator of Sx
    S = Ey*Bz/factor # poynting flux
    # integrate/average over time at each grid point
    sum_t = np.zeros(nx)
    for i in range(timesteps):
        sig = S[:,i]
        indx = np.where(sig > 0) # only care for backward travelling flux
        sig[indx] = 0
        sum_t += sig
    S_t_av = np.abs(sum_t)/timesteps
    # for backward travelling signals, we want to average close to the left-hand boundary
    sum_x = 0
    for i in range(ncells):
        sum_x += S_t_av[i]
    S_av = sum_x/ncells
    if refelctivity:
        return S_av/intensity
    else:
        return S_av

In [ ]:
P_data = np.array()
for dir in dirs:
    P = get_bsrs(dir, ncells = 10, refelctivity=True)
    P_data = np.append(P_data, P)

In [ ]:
P_mean = np.mean(P_data)
P_var = np.mean(P_data)
P_err = 2.0*np.sqrt(P_var)

print(f'Mean reflectivity = {P_mean} W/cm^2')
print(f'Varaiance of reflectivity = {P_var}')
print(f'Error in reflectivity = {P_err} W/cm^2')

In [ ]:
input_file = 'Training_data/train_inputs.json'
output_file = 'Training_data/train_outputs_mean.json'
var_file = 'Training_data/train_outputs_var.json'
train_frac = 0.1

In [ ]:
gp = LPI_GP(input_file=input_file, output_file=output_file,\
            var_file=var_file, train_frac=train_frac)

In [ ]:
gp.set_training_data()

In [ ]:
gp.optimise_noise_GP()

In [ ]:
gp.optimise_GP()

In [ ]:
gp.test_train_plot()

In [ ]:
X_star = np.geomspace(4e14, 4e15, 100)[:,None]
Y_star, V_epi, V_noise = gp.GP_predict(X_star, get_var=True)

In [ ]:
X = np.exp(gp.get_input())
Y = np.exp(gp.get_output())

X_all = np.exp(read_json_file('Training_data/all_inputs.json'))
Y_all = np.exp(read_json_file('Training_data/all_outputs.json'))


In [ ]:
plt.rcParams["figure.figsize"] = [14, 10]

error_epi = 2.0*np.sqrt(V_epi)
error_tot = 2.0*np.sqrt(V_epi + V_noise)

Y_s = Y_star.flatten()
X_s = X_star.flatten()

plt.loglog(X_s, Y_s, color = 'blue', label = 'GP Mean')
plt.fill_between(X_s, (Y_s-error_epi), (Y_s+error_epi), alpha = 0.3, color = 'cyan', label = 'Epistemic Error')
plt.fill_between(X_s, (Y_s-error_tot), (Y_s+error_tot), alpha = 0.15, color = 'red', label = 'Total Error')
plt.plot(X_all, Y_all, 'kx', color = 'red', label = 'All Samples', alpha = 0.8)
plt.plot(X, Y, 'kx', color = 'blue', label = 'Mean Samples')
plt.xlim(4e14, 4e15)
plt.ylim(2e-3, 2e-1)

plt.ylabel(r'Reflectivity - $\mathcal{P}$')
plt.xlabel(r'$I_{L} \,\, W/cm^{2}$')
plt.legend(loc = 0)

In [ ]:
Y_star, V_epi, V_noise = gp.GP_predict(X_star = np.array([intensity])[:,None], get_var=True)
error_epi = 2.0*np.sqrt(V_epi)
error_noise = 2.0*np.sqrt(V_noise)
error_tot = 2.0*np.sqrt(V_epi + V_noise)

In [ ]:
# per_diff_epi = (np.abs((error_epi - P_err))/P_err)*100
# per_diff_noise = (np.abs((error_noise - P_err))/P_err)*100
# per_diff_total = (np.abs((error_tot - P_err))/P_err)*100
P_err = V_noise
err_compare = [P_err, V_epi, V_noise, V_noise]

In [ ]:
# plot scaled sensitivities
fig, ax = plt.subplots(figsize=(10,6))
ax.plot(['Samaple Error', 'GP Epistemic Error', 'GP Noise Error', 'GP Total Error'], err_compare, 'o')
ax.set_xticklabels(['Samaple Error', 'GP Epistemic Error', 'GP Noise Error', 'GP Total Error'], rotation=90)
ax.set_ylabel(r'Error')